In [ ]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  import os
  CWD = '/content/drive/MyDrive/DataSources/SkinCare'
  os.chdir(CWD)
except:None
G_path = './Project_Data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

import sys
#import torchvision.transforms.Compose
import numpy as np
from datetime import datetime	
import random
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import random_split
from torch.utils.data import SubsetRandomSampler
from torch.utils.data import WeightedRandomSampler
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset as dset
import time
from tqdm import tqdm
import pickle as pickle
from statistics import mean
import pandas as pd
pd.options.display.max_colwidth = 250
import sklearn as sk
from sklearn.model_selection import train_test_split
device = torch.device("cuda" if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else "cpu")
from sklearn.utils import resample
import warnings
warnings.filterwarnings('ignore')
import features
from features.usefull_functions import *
from features.NETs import *
from features.Model_Training import *
device = None
try:
    c = torch.cuda.is_available()
    if c:
        device = 'cuda'
        print(f'devis: {device}')

    else:
        try :
            m = torch.backends.mps.is_available()
            if m:
                device = 'mps'
                print(f'devis: {device}')  
        except:    
            device = 'cpu'
            print(f'devis: {device}')          
except:
    None

devis: cuda
devis: cuda


In [ ]:
Binary_classification = True
file = 'Input_DataSet_280x210' 
file = 'Input_DataSet_360x270' 
input_data , labels = pickle.load(open(G_path + '/06_Rescaled_DataSet/'+ file,'rb'))
file = 'Balanced_Test_Set_280x210' 
file = 'Balanced_Test_Set_360x270' 
Evaluation_set = pickle.load(open(G_path + '/06_Rescaled_DataSet/' + file,'rb'))

if Binary_classification:
  labels = labels[1]
  Name = '_Binary_'
else:
  labels = labels[0]
  Name = '_MultiClass_'

In [ ]:
labels[1][:10]

tensor([0, 0, 1, 0, 0, 1, 0, 0, 0, 0])

# **CNN Networks: Configuration**

In [ ]:
net_list = [func for func in dir(CNN_Nets) if callable(getattr(CNN_Nets, func)) and not func.startswith("__")]
X_ = input_data[0][0:5]
for net in net_list:
    print(net)
    model_ = getattr(CNN_Nets,net)
    fc_features = model_().dimention_set(X_)
    print(fc_features)
    model_ = getattr(CNN_Nets,net)
    model_.fc_features = fc_features
    out = model_().forward(X_)
    setattr(CNN_Nets,net,model_)
    # print(out.shape)

Net1
346752
Net10
8512
Net11
2160
Net2
371712
Net3
20160
Net4
20160
Net5
93984
Net6
92928
Net7
768
Net8
768
Net8_a
768
Net8_a_binary
3072
Net8_b
3072
Net8_b_binary
3072
Net9
768


# **Metrics and Performance**

# **Define Training data**

In [ ]:
X_train = input_data[0]
Y_train = labels[0]
X_valid = input_data[1]
Y_valid = labels[1]
X_test = input_data[2]
Y_test = labels[2]

# **Model by Grid**

In [ ]:
Net = [CNN_Nets.Net8_b]
Drop = [0.2,0.22]
LR = [ 0.5e-3,1e-3,1.5e-3]
batch_size = [24]
Momentum = [0.75,0.85,0.9]
epochs = [100]
patience = [20]
weight_decay = [1e-3,1e-5]
loss_func  =  [nn.CrossEntropyLoss]
opt_func = [torch.optim.SGD]

Net = [CNN_Nets.Net8_b_binary]
Drop = [0.28,0.3,0.35]
LR = [ 1.1e-3,0.9e-3,0.7e-3]
batch_size = [24]
Momentum = [0.88]
epochs = [100]
patience = [20]
weight_decay = [1e-3,1e-4, 0.5e-4]
loss_func  =  [nn.CrossEntropyLoss]
opt_func = [torch.optim.SGD]


grid = {
    'Net' : Net
    ,'Drop' : Drop
    ,'LR' : LR
    ,'batch_size' : batch_size
    ,'Momentum' : Momentum
    ,'epochs' : epochs
    ,'patience': patience
    ,'weight_decay' :weight_decay
    ,'loss_func'  :loss_func
    ,'opt_func' : opt_func
}
params = sk.model_selection.ParameterGrid(grid)

In [ ]:
print_epochs = False
save_results = True
save_models = True
cross_valid = False
nr_repeat = 1

shape_ = str(X_train[0].shape[1])+'x'+str(X_train[0].shape[2])
export_name = 'CNN_' + shape_ + Name # {'280_210_','AutoEncoder_'}
kflods = sk.model_selection.KFold(n_splits=10, shuffle=True)
Param_Details, Best_Grid_Model = NNs_grid_searc_cross_valid_trainer(params, X_train ,Y_train, X_valid, Y_valid, X_test, Y_test, kflods, print_epochs, save_results, save_models, cross_valid, nr_repeat, export_name, path=G_path)

   [............................................................] 0/27     

Early Stoppo happen at Epoche 65 after no improvment of 21 epochs 
 => Max_Train_Ac: 87.6%, Max_Valid_Ac: 76.9%, Max_Test_Ac: 77.4%, Min_train_loss: 0.3, Min_test_loss: 0.4  [█.............................] 1/27 [Time => Iter.: 718.4s, Tot.: 718.4s, Remain.: 18677.5s] => last model: Train_Ac: 87.6%, Valid_Ac: 76.9%, Test_Ac: 77.4%, train_loss: 0.3, test_loss: 0.4   

Early Stoppo happen at Epoche 89 after no improvment of 21 epochs 
 => Max_Train_Ac: 91.6%, Max_Valid_Ac: 81.6%, Max_Test_Ac: 80.3%, Min_train_loss: 0.2, Min_test_loss: 0.4  [██............................] 2/27 [Time => Iter.: 981.3s, Tot.: 1699.7s, Remain.: 21246.0s] => last model: Train_Ac: 91.6%, Valid_Ac: 81.6%, Test_Ac: 80.3%, train_loss: 0.2, test_loss: 0.4   

Early Stoppo happen at Epoche 95 after no improvment of 21 epochs 
 => Max_Train_Ac: 93.0%, Max_Valid_Ac: 81.6%, Max_Test_Ac: 80.3%, Min_train_loss: 0.2, Min_test_loss: 0.4  [███....

In [ ]:
plt.subplots(figsize=(8,4))
sns.boxplot(data=model_recalls)
plt.title('Test performance of 100 Cross-Validation',fontsize = 18)
plt.xlabel("Classifiers", fontsize = 14)
plt.ylabel("Accuracy", fontsize = 14)
plt.xticks(fontsize=16, rotation=0)
plt.show()

plt.subplots(figsize=(8,4))
sns.boxplot(data=model_specificity)
plt.title('Valid performance of 100 Cross-Validation',fontsize = 18)
plt.xlabel("Classifiers", fontsize = 14)
plt.ylabel("Accuracy", fontsize = 14)
plt.xticks(fontsize=16, rotation=0)
plt.show()
